In [2]:
!pip3 install pydantic

In [3]:
from pydantic import BaseModel
from typing import Optional
from typing import Any, Dict, List, Optional, OrderedDict, Set, Tuple, Union
from uuid import UUID
from pydantic.networks import AnyUrl

class KeysendCustomRecord(BaseModel):
    podcast: Optional[str]
    feedID: Optional[int]
    url: Optional[AnyUrl]
    guid: Optional[UUID]
    #
    episode: Optional[str]
    itemID: Optional[int]
    episode_guid: Optional[str]
    #
    time: Optional[str]
    ts: Optional[int]
    action: Optional[str] = "stream"
    app_name: Optional[str] = "unknown"
    app_version: Optional[str]
    boost_link: Optional[str]
    message: Optional[str]
    name: Optional[str]
    pubkey: Optional[str]
    sender_key: Optional[str]
    sender_name: Optional[str]
    sender_id: Optional[str]
    sig_fields: Optional[str]
    signature: Optional[str]
    speed: Optional[str]
    uuid: Optional[str]
    value_msat: Optional[int]
    cr_value_msat: Optional[int]
    value_msat_total: Optional[int]



In [4]:
def b64_hex_transform(plain_str: str) -> str:
    """Returns the b64 transformed version of a hex string"""
    a_string = bytes.fromhex(plain_str)
    return base64.b64encode(a_string).decode()


In [5]:

def b64_transform(plain_str: str) -> str:
    """Returns the b64 transformed version of a string"""
    return base64.b64encode(plain_str.encode()).decode()


In [6]:
def utf8len(s):
    return len(s.encode("utf-8"))



In [7]:

def utf8len_dict(a_dict: dict):
    s = json.dumps(a_dict)
    return len(s.encode("utf-8"))


In [8]:

async def local_pay_keysend(
    amt: int,
    custom_record: KeysendCustomRecord,
    dest_pubkey: str = Config.MY_UMBREL_PUBLIC_KEY,
    hive_accname: str = "brianoflondon",
    add_text: int = 0,
):
    """Pay a keysend invoice using the chosen node"""
    headers, cert = get_lnd_headers_cert(admin=True)

    if add_text:
        extra_text = token_hex(add_text)
        extra_text = " " + extra_text
        custom_record = get_custom_record(action="boost", extra_text=extra_text)

    # Base 64 encoded destination bytes
    dest = b64_hex_transform(dest_pubkey)
    cust_rec_str = json.dumps(custom_record.dict(exclude_unset=True, exclude_none=True))

    logging.info(
        json.dumps(custom_record.dict(exclude_unset=True, exclude_none=True), indent=2)
    )

    pre_image = token_hex(32)
    payment_hash = sha256(bytes.fromhex(pre_image))

    dest_custom_records = {
        5482373484: b64_hex_transform(pre_image),
        34349334: b64_transform("Go Podcasting!"),
        7629169: b64_transform(cust_rec_str),
    }

    if hive_accname:
        dest_custom_records["818818"] = b64_transform(hive_accname)

    url = f"{Config.LOCAL_LND_NODE_ADDRESS}v1/channels/transactions"
    data = {
        "dest": dest,
        "amt": amt,
        "payment_hash": b64_hex_transform(payment_hash.hexdigest()),
        "dest_custom_records": dest_custom_records,
    }
    # data = {"dest": dest, "amt": amt}
    logging.info(
        f"Paying Keysend for {amt:,} sats to: {dest_pubkey} | "
        f"Keysend size: {utf8len_dict(dest_custom_records)}"
    )
    try:
        async with httpx.AsyncClient(verify=cert) as client:
            response = await client.post(
                url=url, headers=headers, data=json.dumps(data)
            )
    except Exception as ex:
        logging.error(f"Keysend Error: {ex} {ex.__class__}")
        return

    logging.info(json.dumps(response.json(), indent=2))
    return response


NameError: name 'Config' is not defined

In [ ]:


def get_custom_record(
    time_s: int = 59,
    iteration: int = None,
    action: str = "stream",
    extra_text: str = None,
):
    """Return the custom record for testing"""
    custom_record = KeysendCustomRecord(
        action=action,
        app_name="V4Vapp Hive Bridge",
        # app_name="v4vapi-test-production",
        podcast="blocktvnews Podcast on Hive",
        name="blocktvnews via 3Speak",
        url="https://3speak.tv/rss/blocktvnews.xml",
        episode="THE DAPP SHOW - EPISODE 2 - DECENTRALAND & B2EXPAND",
        episode_guid="hive-100421/@blocktvnews/uxjnnxsp",
        itemID="3139296827",
        ts=time_s,
        speed="1",
        feedID="4100216",
        message=f"test script - iteration {iteration}{extra_text}",
    )
    return custom_record
